In [3]:
from data import load_data_file

train_data = load_data_file("../data/GenBench/nonstory")

In [4]:
from data import create_gloss_vocab
from uspanteko_morphology import morphology

glosses = create_gloss_vocab(morphology)

[gloss for gloss in set([gloss for row in train_data for gloss in row.gloss_list(segmented=True)]) if
 gloss not in glosses]

[]

In [12]:
glosses

['???',
 '[SEP]',
 'A1P',
 'A2P',
 'A1S',
 'A2S',
 'E1',
 'E2',
 'E3',
 'E1P',
 'E2P',
 'E3P',
 'E1S',
 'E2S',
 'E3S',
 'AFE',
 'ITR',
 'TRN',
 'AP',
 'APLI',
 'CAU',
 'MOV',
 'PAS',
 'REC',
 'RFX',
 'COM',
 'INC',
 'PRG',
 'COND',
 'IMP',
 'INT',
 'TAM',
 'DIR',
 'PP',
 'SC',
 'SV',
 'EXS',
 'POS',
 'VI',
 'VT',
 'ADJ',
 'NOM',
 'PRON',
 'S',
 'SAB',
 'TOP',
 'VOC',
 'CLAS',
 'DIM',
 'GNT',
 'NUM',
 'AGT',
 'INS',
 'PL',
 'ADV',
 'AFI',
 'NEG',
 'PART',
 'SREL',
 'ART',
 'DEM',
 'CONJ',
 'ENF',
 'ITS',
 'MED',
 'PREP']